## Read and visualize DICOM-RTSTRUCT images with pydicom

In [3]:
%matplotlib inline

import numpy as np # linear algebra
import pydicom
import gdcm
import os
import cv2
import scipy.ndimage
import matplotlib.pyplot as plt
import random
from IPython.display import Image
    

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


### DICOM-RTSTRUCT Contour Data

Contours drawn for radiotherapy are saved as DICOM RT Structure Set (“RT” stands for radiotherapy.) in DICOM Standard, and usually as a single file. You can locate this file among CT or MRI data sets quite reliably, by traversing recursively through the directories and looking for MODALITY of “RTSTRUCT”.

The contours are saved as 2D polygons slice by slice under the Contour Data (3006,0050) tag, which species the data format as:

Sequence of (x,y,z) triplets defining a contour in the patient based coordinate system described in Section C.7.6.2.1.1 (mm). See Section C.8.8.6.1 and Section C.8.8.6.3.

In [4]:
# Example of DICOM RT Structure Set file
example_dcm_structure = '../data/rt_structure_ex/TCGA-HNSC/TCGA-CV-5966/11-19-1994-NECKASG-25416/1.000000-83598/1-1.dcm'


In [5]:
ds = pydicom.read_file(example_dcm_structure)
ds

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20040925'
(0008, 0013) Instance Creation Time              TM: '141514'
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.1706.4009.198949163998597526872719573002
(0008, 0020) Study Date                          DA: '19941119'
(0008, 0030) Study Time                          TM: ''
(0008, 0050) Accession Number                    SH: '2819497684894126'
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'ADAC'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1030) Study Description                   LO: 'NECK/ASG'
(0008, 1090) Manufacturer's Model Name           LO: 'Pinnacle3'
(0010, 0010) Patient's Name                      PN: 'TCGA-CV-5966'
(0010, 0020) Patient ID                    

The contours are saved as 2D polygons slice by slice under the Contour Data (3006,0050) tag, which species the data format as:

Sequence of (x,y,z) triplets defining a contour in the **patient based coordinate system** described in Section C.7.6.2.1.1 (mm). See Section C.8.8.6.1 and Section C.8.8.6.3.

In [20]:
ds.ROIContourSequence[0].ContourSequence[0].ContourData

[240.933, 171.331, -1362.43]

### Problems

- How to Match Contours (ROI) to DICOM files
- Map from CT coordinates to patient coordinates

### Show example

In [ ]:
#Read RT Struct
# https://dicom.innolitics.com/ciods/ct-image/image-plane/00200037
# https://dicom.innolitics.com/ciods/rt-structure-set/roi-contour/30060039/30060040/30060050

def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(15, 15))
    plt.imshow(image, cmap = 'gray')
#    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()


path = '/media/kamila/System/Users/Kama/Documents/DATASETS/DICOM/HNSCC/HNSCC-4series/HNSCC-01-0002/11-20-2001-RT SIMULATION-57976/2.000000-57922/'
rt_struct_file = path + '1-1.dcm'

def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(15, 15))
    plt.imshow(image, cmap = 'gray')
#    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

ds = pydicom.dcmread(rt_struct_file)

print(ds.dir())
print(ds.SOPClassUID)
    

if (ds.SOPClassUID == "1.2.840.10008.5.1.4.1.1.481.3"):
        
    print('\n')
    print('Structures len = ', len(ds.StructureSetROISequence))
    print('ROIs len = ',len(ds.ROIContourSequence))
    print('\n')
    structID = 15
    
    
    ROI = [ds.ROIContourSequence[u] for u in range(len(ds.ROIContourSequence)) 
           if ds.ROIContourSequence[u].ReferencedROINumber == ds.StructureSetROISequence[structID].ROINumber][0]

    print(ds.StructureSetROISequence[structID].ROINumber,ROI.ReferencedROINumber)
    print('\n')
    print(ds.StructureSetROISequence[structID])
    print('\n')
#     print(ROI)
#     print(ROI.dir())

    for seq in ROI.ContourSequence:
        #print(seq.ContourImageSequence[0][0x0008, 0x1155].value)               # SOP Instance UID Attribute
        #print(seq.ContourImageSequence[0][0x0008, 0x1155].value)               # CT image
        #print(seq.ContourData)                                                 # Contour points
        im_d = seq.ContourImageSequence[0][0x0008, 0x1155].value
#         dicImage = pydicom.dcmread(path+'CT/CT.' + seq.ContourImageSequence[0][0x0008, 0x1155].value + '.dcm')
        dicImage = pydicom.dcmread(path + im_d + '.dcm')
#         print(dicImage[0x0020, 0x0037].value)       # Image orientation
#         print(dicImage[0x0020, 0x0032].value)       # Image origin
#         print(dicImage[0x0028, 0x0030].value)       # Pixel spacing

        # coordinates -> pixels
        # pixel_coords = [(np.ceil((x - origin_x) / x_spacing), np.ceil((y - origin_y) / y_spacing))  for x, y, _ in coord]
        M = np.zeros((3,3),dtype = np.float32)
        M[0,0] = dicImage[0x0020, 0x0037].value[1]* dicImage[0x0028, 0x0030].value[0]
        M[1,0] = dicImage[0x0020, 0x0037].value[0]* dicImage[0x0028, 0x0030].value[0]
        M[0,1] = dicImage[0x0020, 0x0037].value[4]* dicImage[0x0028, 0x0030].value[1]
        M[1,1] = dicImage[0x0020, 0x0037].value[3]* dicImage[0x0028, 0x0030].value[1]
        M[0,2] = dicImage[0x0020, 0x0032].value[0]
        M[1,2] = dicImage[0x0020, 0x0032].value[1]
        M[2,2] = 1.0
        M = np.linalg.inv(M)
        points = np.swapaxes(np.reshape(seq.ContourData,(-1,3)),0,1)
        points[2,:].fill(1)
        points = np.dot(M,points)[:2,:]
#        print(points)
        
        big = int(np.max(dicImage.pixel_array))
#        print(big)
        background = np.zeros((dicImage.pixel_array.shape),dtype = np.float32)
        for id in range(points.shape[1]-1):
            cv2.line(background,(int(points[1,id]),int(points[0,id])),(int(points[1,id+1]),int(points[0,id+1])),big,1)
#            dicImage.pixel_array[int(points[0,id]),int(points[1,id])] = big-id*40
        cv2.line(background,(int(points[1,points.shape[1]-1]),int(points[0,points.shape[1]-1])),(int(points[1,0]),int(points[0,0])),big,1)
    
    
        for id in range(points.shape[1]-1):
            cv2.line(dicImage.pixel_array,(int(points[1,id]),int(points[0,id])),(int(points[1,id+1]),int(points[0,id+1])),big,1)
#            dicImage.pixel_array[int(points[0,id]),int(points[1,id])] = big-id*40
        cv2.line(dicImage.pixel_array,(int(points[1,points.shape[1]-1]),int(points[0,points.shape[1]-1])),(int(points[1,0]),int(points[0,0])),big,1)
        
#        print(seq.dir())
#        print(seq.ContourGeometricType)
        showOpencvImage(dicImage.pixel_array)
        showOpencvImage(background)
#         mask = cv2.fillPoly(img=background, pts=points, color=1)
#         mask = np.array([scn.binary_fill_holes(y) if y.max() == 1 else y for y in background])
#         mask_filled = fill_contour(mask)
#         showOpencvImage(mask_filled)
#         showOpencvImage(big)
#         imdddd = str(seq.ContourImageSequence[0][0x0008, 0x1155].value)
#         print(imdddd)
#         print(type(imdddd))
        break

### Saving contours as numpy arrays and masks

#### Change file names and save RT STRUCT among other CT images in one folder




In [ ]:
MAIN_PATH = '/media/kamila/System/Users/Kama/Documents/DATASETS/DICOM/HNSCC/HNSCC-4series'
DICOM_FILENAMES_LIST = []
for dirName, subdirList, fileList in os.walk(MAIN_PATH):
        if len(fileList) > 1:
            DICOM_FILENAMES_LIST.append(dirName)

def return_ct_imgs_folder():
    pass

def return_rt_struct_folder():
    pass

def change_filenames_acc_to_SOPInstUIDAttr(dioms_folder):
    for dcm_file in os.listdir(dioms_folder):
        dicom_name_full = os.path.join(dioms_folder, dcm_file)
        ds = dicom.read_file(dicom_name_full)
        new_name = ds[0x08,0x18].value + '.dcm' # SOP Instance UID Attribute
        new_name_full = os.path.join(dioms_folder, new_name)
        os.rename(dicom_name_full, new_name_full)

def copy_rtstruct_to_correct_dicomsct_folder(main_path = MAIN_PATH):
    # function arguments: main_path, list_of_dicom_folders
    for dirName, subdirList, fileList in os.walk(MAIN_PATH):
            if len(fileList) == 1:
                contour_file = get_contour_file(dirName)
                if contour_file:
                    print(dirName + '/' + contour_file)
                    # copy it to dicom file
                    
                    
def clean_data():
    pass


def split_train_test():
    pass


def return_structID_segName_maping():
    pass

In [ ]:
def slice_order_by_instnumber(path):
    """
    Takes path of directory that has the DICOM images and returns
    a ordered list that has ordered filenames
    Inputs
        path: path that has .dcm images
    Returns
        ordered_slices: ordered tuples of filename and z-position
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    slices = []
    for s in os.listdir(path):
        try:
            f = pydicom.read_file(path + '/' + s)
            f.pixel_array  # to ensure not to read contour file
            slices.append(f)
        except:
            continue
            
    slices.sort(key = lambda x: int(x.InstanceNumber))

    slice_dict_sorted = [(s.SOPInstanceUID, s.ImagePositionPatient[-1]) for s in slices]
#     ordered_slices = sorted(slice_dict.items(), key=operator.itemgetter(1))
    return slice_dict_sorted


# from: https://github.com/KeremTurgutlu/dicom-contour
def get_contour_file(path):
    """
    Get contour file from a given path by searching for ROIContourSequence 
    inside dicom data structure.
    More information on ROIContourSequence available here:
    http://dicom.nema.org/medical/dicom/2016c/output/chtml/part03/sect_C.8.8.6.html
    
    Inputs:
            path (str): path of the the directory that has DICOM files in it, e.g. folder of a single patient
    Return:
        contour_file (str): name of the file with the contour
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    # get .dcm contour file
    fpaths = [path + f for f in os.listdir(path) if '.dcm' in f]
    n = 0
    contour_file = None
    for fpath in fpaths:
        f = pydicom.read_file(fpath)
        if 'ROIContourSequence' in dir(f):
            contour_file = fpath.split('/')[-1]
            n += 1
    if n > 1: warnings.warn("There are multiple contour files, returning the last one!")
    if contour_file is None: print("No contour file found in directory")
    return contour_file


def get_img_mask_dict_multi(path, structIDs, colormap, rt_struct_file = None):
    pass


def get_img_mask_dict_binary(structID, path, rt_struct_file = None):
    if rt_struct_file == None:
        rt_struct_file = get_contour_file(path)
        if path[-1] != '/': path += '/'
        rt_struct_file = path + rt_struct_file
    ds = pydicom.dcmread(rt_struct_file)
    contour_dict = {}
# if (ds.SOPClassUID == "1.2.840.10008.5.1.4.1.1.481.3"):  
    
    ROI = [ds.ROIContourSequence[u] for u in range(len(ds.ROIContourSequence)) 
           if ds.ROIContourSequence[u].ReferencedROINumber == ds.StructureSetROISequence[structID].ROINumber][0]

    for seq in ROI.ContourSequence:
        imgid_ = seq.ContourImageSequence[0][0x0008, 0x1155].value
        dicImage = pydicom.dcmread(path + imgid_ + '.dcm')
        M = np.zeros((3,3),dtype = np.float32)
        M[0,0] = dicImage[0x0020, 0x0037].value[1]* dicImage[0x0028, 0x0030].value[0]
        M[1,0] = dicImage[0x0020, 0x0037].value[0]* dicImage[0x0028, 0x0030].value[0]
        M[0,1] = dicImage[0x0020, 0x0037].value[4]* dicImage[0x0028, 0x0030].value[1]
        M[1,1] = dicImage[0x0020, 0x0037].value[3]* dicImage[0x0028, 0x0030].value[1]
        M[0,2] = dicImage[0x0020, 0x0032].value[0]
        M[1,2] = dicImage[0x0020, 0x0032].value[1]
        M[2,2] = 1.0
        M = np.linalg.inv(M)
        points = np.swapaxes(np.reshape(seq.ContourData,(-1,3)),0,1)
        points[2,:].fill(1)
        points = np.dot(M,points)[:2,:]

        points_collection = []
        for id in range(points.shape[1]-1):
            points_collection.append([ int(points[1,id]),int(points[0,id]) ])

        points_collection_corr = []
        points_collection_corr.append(points_collection)
        points_collection_corr = np.array(points_collection_corr)

        mask_background = np.zeros((dicImage.pixel_array.shape),dtype = np.float32)
        mask_ = cv2.fillPoly(mask_background, points_collection_corr, 255)
        img_ = dicImage.pixel_array
        
        contour_dict[imgid_] = [np.array(img_), np.array(mask_)]

    return contour_dict



def collect_dataset_single_patient(structID, path, rt_struct_file = None):
    """
    Create dataset of numpy arrays: image + correcponding mask.

    
    Contour - collection of coordinates - from RTStructure file is chosen by structID number, 
    then transporm to numpy array binary mask.
    
    E.g. rt_file can have contours for different parts of the brain 
    such as ventricles, tumor, etc...
    
    You can use get_roi_names to find which index to use
    
    Inputs:
        path (String): path to folder that contains dicom ct images
        rt_file (String or None): path to dicom RTSTRUCT file. 
                If None, means RTSTRUCTc file should be found inside main folder - path,
                among others dicom ct images.
        index (int): Index for ROI Sequence
    Return:
        images, masks (numpy arrays): collection of images and corresponding masks in correct order
    """
    
    images = []
    masks = []
    # get slice orders
    ordered_slices = slice_order_by_instnumber(path)
    # get dict containing prepared imgs and masks - in random order though
    img_mask_dict = get_img_mask_dict_binary(structID = structID, path = path, rt_struct_file = rt_struct_file)
    if path[-1] != '/': path += '/'
    for k,v in ordered_slices:
        # get data from img-mask dict
        if k in img_mask_dict:
            images.append(img_mask_dict[k][0])
            masks.append(img_mask_dict[k][1])
        # get data from dicom.read_file
        else:
            img_arr = pydicom.read_file(path + k + '.dcm').pixel_array
            mask_arr = np.zeros_like(img_arr)
            images.append(img_arr)
            masks.append(mask_arr)
    return np.array(images), np.array(masks)


def get_dataset_fulldicom(structID, list_of_folders):
    data_out = []
    shapes = {}
    for folder in list_of_folders:
        voxels, masks = collect_dataset_single_patient(structID=structID, path=folder)
        if voxels.shape in shapes:
            shapes[voxels.shape] += 1
        else:
            shapes[voxels.shape] = 1
            # Saves data
        data_out.append((voxels, masks))
    print(shapes)
    return data_out

Sources

[1] https://zhangresearch.org/post/parse-dicom-rtstruct-into-binary-masks/

[2] http://aapmchallenges.cloudapp.net/forums/3/2/

[3] 

[4] 

[5] 